# Notebook to evaluate Benchmark Performance

In [1]:
import datetime

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import pyfolio as pf

from deep_rl_asset_allocation.utils import zipline_utils

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# "Agg" backend is for writing to file, not for rendering in a window
matplotlib.use('Agg')

# plotting in notebook
%matplotlib inline

In [4]:
# print options for pandas
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [5]:
%load_ext zipline

### Zipline: Buy & Hold

In [ ]:
%%zipline --start 2016-1-4 --end 2020-5-7 --capital-base 1000000.0 --bundle djia-csvdir-bundle --no-benchmark -o benchmark_buy_n_hold.pkl 

# imports
from zipline.api import order, symbol, record
from zipline.finance import commission

selected_stock = 'DJIA'
n_stocks_to_buy = 10

def initialize(context):
    context.has_ordered = False  

def handle_data(context, data):
    # record price for further inspection
    record(price=data.current(symbol(selected_stock), 'price'))
    
    # trading logic
    if not context.has_ordered:
        # placing order, negative number for sale/short
        order(symbol(selected_stock), n_stocks_to_buy)
        # setting up a flag for holding a position
        context.has_ordered = True

In [ ]:
# read pkl file
benchmark_df = pd.read_pickle('benchmark_buy_n_hold.pkl')
# print(f"{benchmark_df.columns}")

# get transactions
transactions_list = []
for transactions in benchmark_df.transactions.values:
    if transactions != []:
        for transaction in transactions:
            transactions_list.append(transaction)
transactions = pd.DataFrame.from_records(transactions_list)
print(transactions)

# get teardown
returns, positions, transactions = pf.utils.extract_rets_pos_txn_from_zipline(benchmark_df)
benchmark_perf = zipline_utils.get_performance_summary(returns)
print(f'\nPerformance:\n{benchmark_perf}')

# plot results
zipline_utils.visualize_results(df=benchmark_df, title="Benchmark")

### Zipline: 1/n Benchmark

In [ ]:
%%zipline --start 2016-1-4 --end 2020-5-7 --capital-base 1000000.0 --bundle djia-csvdir-bundle --no-benchmark -o benchmark_1_n.pkl 

from zipline.api import order_percent, symbols
from zipline.finance import commission
import numpy as np 
import pandas as pd

def initialize(context):
    context.set_commission(commission.PerShare(cost=0.0, min_trade_cost=0))
    context.assets = symbols(
        'AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DD', 'DIS', 'GS', 'HD',
        'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE',
        'PFE', 'PG', 'RTX', 'TRV', 'UNH', 'V', 'VZ', 'WBA', 'WMT', 'XOM',
    )
    context.n_assets = len(context.assets)
    context.has_position = False

def handle_data(context, data):
    if not context.has_position:
        for asset in context.assets:
            order_percent(asset, 1/context.n_assets)
        context.has_position = True

In [ ]:
# read pkl file
benchmark_df = pd.read_pickle('benchmark_1_n.pkl')
# print(f"{benchmark_df.columns}")

# get transactions
transactions_list = []
for transactions in benchmark_df.transactions.values:
    if transactions != []:
        for transaction in transactions:
            transactions_list.append(transaction)
transactions = pd.DataFrame.from_records(transactions_list)
print(transactions)

# get teardown
returns, positions, transactions = pf.utils.extract_rets_pos_txn_from_zipline(benchmark_df)
benchmark_perf = zipline_utils.get_performance_summary(returns)
print(f'\nPerformance:\n{benchmark_perf}')

# plot results
zipline_utils.visualize_results(df=benchmark_df, title="Benchmark")